In [25]:
import pandas as pd
import os
import xml.etree.cElementTree as ET
import re
import numpy as np

In [77]:
d =[1,2,3,4,5]
df = pd.DataFrame(data=d, index = ['one','two','one','two','two'], columns = ['label'])
df.head()

,label
one,1
two,2
one,3
two,4
two,5


In [15]:
files = os.listdir("./coropus_for_korzun/")
files.sort()
Time = '{http://www.abbyy.com/ns/Time#}'
rdfs = '{http://www.w3.org/2000/01/rdf-schema#}'
Aux = '{http://www.abbyy.com/ns/Aux#}'

In [50]:
stripv = np.vectorize(lambda x,a: x.strip(a))

In [57]:
def split_text(s):
    s = s.replace('\u2028', ' ').split(' ')
    s = list(filter(lambda x: x !='', s))
    s = stripv(stripv(s,'.'),',')
    return s

In [98]:
WORDS = []
INDEX = []
for fl in files:  
    XML_FILE = os.path.join("./coropus_for_korzun/", fl)
    tree = ET.ElementTree(file = XML_FILE)
    root = tree.getroot()
    dates = set()
    for child in root:
        if child.tag == Time + 'PointOfTime':
            children = child.getchildren()        
            flag = False;       
            for part in children:            
                if part.tag == Time + 'month' or part.tag == Time + 'day' or part.tag == Time + 'year':
                    flag = True
                    break
            if flag == True:
                label = child.find(rdfs+'label')           
                if label != None:                   
                    dates.add(label.text) 
                else:
                    dates.add(children[0].text)
    text = root.find(Aux+'TextAnnotations').find(Aux+'document_text').text

    words = split_text(text)

    splitted_dates = []
    for date in dates:
        splitted_dates.append(split_text(date))    

    indexes = len(words)*['NONE']

    for i in range(len(words)):
        for j in range(len(splitted_dates)):
            if words[i] == splitted_dates[j][0]:
                if len(splitted_dates[j]) == 1:
                    indexes[i] = 'D_ST'
                else:    
                    flag = True
                    for k in range(len(splitted_dates[j])):
                        try:
                            if words[i+k] != splitted_dates[j][k]:
                                flag = False
                        except Exception as e:
                            flag = False
                    if flag == True:
                        indexes[i] = 'D_ST'
                        for k in range(len(splitted_dates[j]) - 1 ):
                            indexes[i+1+k] = 'D_PT'
                            
    WORDS.append(words)
    INDEX.append(indexes) 


                      

In [109]:
kek = []
for a in WORDS:
    for b in a:
        kek.append(b)

In [110]:
kek2 = []
for a in INDEX:
    for b in a:
        kek2.append(b)

In [111]:
len(kek2)

459952

In [112]:
len(kek)

459952

In [113]:
df = pd.DataFrame(data=kek, index = kek2, columns = ['label'])                        
df.to_csv('dates.csv')  